In [3]:
'''
Code intended to evaluate the performances of Orbslam3 and DSO using custom dataset in TUM format.
Requirements.
1) Container containing DSO (custom)
2) Container containing ORBSLAM3 (custom)
3) Folder containing the datasets
4) Evo tools ;D
'''

import docker
import os
import sys
import numpy as np
import pandas as pd
import yaml

In [9]:
# Load config
root_path = os.getcwd()
config_path = os.path.join(root_path, "config")
docker_config = os.path.join(config_path, "docker_containers.yaml")
local_config = os.path.join(config_path, "local_host_params.yaml")
print("Current config path: {}".format(config_path))
try:
    with open(docker_config, "r") as yamlfile:
        docker_data_cfg = yaml.load(yamlfile, Loader=yaml.FullLoader)
        print("\t -Docker config loaded;")
except OSError:
    print("Could not open/read file:", docker_config)
    sys.exit()

try:
    with open(local_config, "r") as yamlfile:
        local_data_cfg = yaml.load(yamlfile, Loader=yaml.FullLoader)
        print("\t -Local config loaded;")
except OSError:
    print("Could not open/read file:", local_config)
    sys.exit()

Current config path: /home/croc/Documenti/Repositories/evoTools/notebooks/config
	 -Docker config loaded;
	 -Local config loaded;


In [13]:
docker_data_cfg

{'Containers': [{'DSO': {'Id': '5b2c72b08226',
    'Names': 'test_dso',
    'datasetFolder': '/root/Archive/Dataset',
    'binPath': '/root/Programs/dso/build/bin/'}},
  {'ORBSLAM3': {'Id': '4ecad379b081',
    'Names': 'test_orbslam3',
    'datasetFolder': '/root/Archive/Dataset',
    'binPath': '/root/Programs/ORB_SLAM3/Examples/Monocular/'}}]}